In [1]:
import torch
import numpy as np
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import pandas as pd

In [594]:
import torch.nn.functional as F

In [1030]:
###  Load and Clean Data  ###

#Load as panda
xy_train = pd.read_csv('./data/titanic/train.csv')
x_test = pd.read_csv('./data/titanic/test.csv')
#xy_train = pd.read_csv('./data/titanic/debug_1s.csv')

#Scale Sex
xy_train['Sex'] = xy_train['Sex'].map({'male':1, 'female':0})
x_test['Sex'] = x_test['Sex'].map({'male':1, 'female':0})


#Pclass to one hot
pclass_dummies_tr = pd.get_dummies(xy_train['Pclass'], prefix='Pclass')
xy_train = pd.concat([xy_train, pclass_dummies_tr], axis=1)

pclass_dummies_t = pd.get_dummies(x_test['Pclass'], prefix='Pclass')
x_test = pd.concat([x_test, pclass_dummies_t], axis=1)

#Scale Fare
#xy_train.Fare = xy_train.Fare/100
#x_test.Fare = x_test.Fare/100



xy_train.sample(3)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_1,Pclass_2,Pclass_3
387,388,1,2,"Buss, Miss. Kate",0,36.0,0,0,27849,13.0000,NaN,S,0,1,0
448,449,1,3,"Baclini, Miss. Marie Catherine",0,5.0,2,1,2666,19.2583,NaN,C,0,0,1
452,453,0,1,"Foreman, Mr. Benjamin Laventall",1,30.0,0,0,113051,27.7500,C111,C,1,0,0


In [1031]:
#Prep Validation Set [should have done this a long time ago!!!!!]
temp_validation_set = pd.read_csv('./data/titanic/full_titanic3_validation.csv')
validation_set = pd.read_csv('./data/titanic/test.csv')


df = pd.DataFrame(columns=['survived'])

#for each ticket in validation_set
for iv, rv in validation_set.iterrows():
    vt = (rv[8])
    #print ('vt is: ' +str (vt))
    #print (temp_validation_set.iloc[1])
    vs = (temp_validation_set['survived'].loc[temp_validation_set['ticket'] == vt])
    #print (vs.values[0])
    df = df.append([{'survived':(vs.values[0]).astype(int)}], ignore_index=True)
    #print (type(vs.values[0]))
    
    
validation_set = pd.concat([validation_set, df], axis=1)

validation_set.describe()   
validation_set.sample(10)
validation_set.head(n=10)

,PassengerId,Pclass,Name,Sex,Survive,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,survived
0,892,3,"Kelly, Mr. James",male,NaN,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,1.0,47.0,1,0,363272,7.0000,NaN,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,NaN,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,NaN,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,1.0,22.0,1,1,3101298,12.2875,NaN,S,1
5,897,3,"Svensson, Mr. Johan Cervin",male,NaN,14.0,0,0,7538,9.2250,NaN,S,1
6,898,3,"Connolly, Miss. Kate",female,0.0,30.0,0,0,330972,7.6292,NaN,Q,0
7,899,2,"Caldwell, Mr. Albert Francis",male,NaN,26.0,1,1,248738,29.0000,NaN,S,1
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,NaN,18.0,0,0,2657,7.2292,NaN,C,1
9,901,3,"Davies, Mr. John Samuel",male,NaN,21.0,2,0,A/4 48871,24.1500,NaN,S,0


In [950]:
print (validation_set.dtypes)
print (temp_validation_set.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Survive        float64
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object
pclass       float64
survived     float64
name          object
sex           object
age          float64
sibsp        float64
parch        float64
ticket        object
fare         float64
cabin         object
embarked      object
boat          object
body         float64
home.dest     object
dtype: object


In [1063]:
#age cleanup WIP
grouped_train = xy_train.groupby(['Sex','Pclass','Title'])
grouped_median_train = grouped_train.median()
grouped_median_train = grouped_median_train.reset_index()[['Sex', 'Pclass', 'Title', 'Age']]
grouped_median_train.head()

,Sex,Pclass,Title,Age
0,0,1,Miss,30.0
1,0,1,Mrs,40.0
2,0,1,Officer,49.0
3,0,1,Royalty,40.5
4,0,2,Miss,24.0


In [1064]:
#age cleanup part 2
def fill_age(row):
    condition = (
        (grouped_median_train['Sex'] == row['Sex']) & 
        (grouped_median_train['Title'] == row['Title']) & 
        (grouped_median_train['Pclass'] == row['Pclass'])
    ) 
    return grouped_median_train[condition]['Age'].values[0]

xy_train['Age'] = xy_train.apply(lambda row: fill_age(row) if np.isnan(row['Age']) else row['Age'], axis=1)
x_test['Age'] = x_test.apply(lambda row: fill_age(row) if np.isnan(row['Age']) else row['Age'], axis=1)

In [1065]:
#optional age cleanup part 3 (scale)
xy_train.Age = xy_train.Age/80
x_test.Age = x_test.Age/80

In [1033]:
Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}


# we extract the title from each name
xy_train['Title'] = xy_train['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
x_test['Title'] = x_test['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())

# a map of more aggregated title
# we map each title
xy_train['Title'] = xy_train.Title.map(Title_Dictionary)
x_test['Title'] = x_test.Title.map(Title_Dictionary)

#convert to one hot
titles_dummies_tr = pd.get_dummies(xy_train['Title'], prefix='Title')
xy_train = pd.concat([xy_train, titles_dummies_tr], axis=1)

titles_dummies_t = pd.get_dummies(x_test['Title'], prefix='Title')
x_test = pd.concat([x_test, titles_dummies_t], axis=1)


xy_train.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Pclass_1,Pclass_2,Pclass_3,Title,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
46,47,0,3,"Lennon, Mr. Denis",1,NaN,1,0,370371,15.5000,...,0,0,1,Mr,0,0,1,0,0,0
96,97,0,1,"Goldschmidt, Mr. George B",1,71.00,0,0,PC 17754,34.6542,...,1,0,0,Mr,0,0,1,0,0,0
831,832,1,2,"Richards, Master. George Sibley",1,0.83,1,1,29106,18.7500,...,0,1,0,Master,1,0,0,0,0,0


In [1034]:
xy_train['FamilyS'] = xy_train['SibSp'] + xy_train['Parch'] + 1
x_test['FamilyS'] = x_test['SibSp'] + x_test['Parch'] + 1



# introducing a new feature : the size of families (including the passenger)
xy_train['FamilySize'] = xy_train['Parch'] + xy_train['SibSp'] + 1
   
# introducing other features based on the family size
xy_train['Singleton'] = xy_train['FamilySize'].map(lambda s: 1 if s == 1 else 0)
xy_train['SmallFamily'] = xy_train['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
xy_train['LargeFamily'] = xy_train['FamilySize'].map(lambda s: 1 if 5 <= s else 0)


# introducing a new feature : the size of families (including the passenger)
x_test['FamilySize'] = x_test['Parch'] + x_test['SibSp'] + 1
   
# introducing other features based on the family size
x_test['Singleton'] = x_test['FamilySize'].map(lambda s: 1 if s == 1 else 0)
x_test['SmallFamily'] = x_test['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
x_test['LargeFamily'] = x_test['FamilySize'].map(lambda s: 1 if 5 <= s else 0)



In [854]:
xy_train.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S,Pclass_fit,Fare_fit
255,256,1,3,"Touma, Mrs. Darwis (Hanne Youssef Razi)",0,0.3625,0,2,2650,15.2458,...,0,0,0,0,1,1,0,0,0.827377,-0.341452
637,638,0,2,"Collyer, Mr. Harvey",1,0.3875,1,1,C.A. 31921,26.2500,...,0,0,0,0,1,0,0,1,-0.369365,-0.119886
419,420,0,3,"Van Impe, Miss. Catharina",0,0.1250,0,2,345773,24.1500,...,0,0,0,0,1,0,0,1,0.827377,-0.162169


In [227]:
#process fare
xy_train.Fare.fillna(xy_train.Fare.mean(), inplace=True)
x_test.Fare.fillna(xy_train.Fare.mean(), inplace=True) # we use tran set data here



In [851]:
#extra fare processing
#xy_train.Fare = xy_train.Fare/513
#x_test.Fare = x_test.Fare/513

#put back
xy_train.Fare = xy_train.Fare*513
x_test.Fare = x_test.Fare*513

In [853]:
# test : scale fare
xy_train['Fare_fit'] = StandardScaler().fit_transform(xy_train['Fare'].values.reshape(-1, 1))
x_test['Fare_fit'] = StandardScaler().fit_transform(x_test['Fare'].values.reshape(-1, 1))

In [1069]:
#process cabin

# replacing missing cabins with U (for Uknown)
xy_train.Cabin.fillna('U', inplace=True)
x_test.Cabin.fillna('U', inplace=True)
    
# mapping each Cabin value with the cabin letter
xy_train['Cabin'] = xy_train['Cabin'].map(lambda c: c[0])
x_test['Cabin'] = x_test['Cabin'].map(lambda c: c[0])
    
# dummy encoding ...
cabin_dummies_tr = pd.get_dummies(xy_train['Cabin'], prefix='Cabin')    
xy_train = pd.concat([xy_train, cabin_dummies_tr], axis=1)

cabin_dummies_t = pd.get_dummies(x_test['Cabin'], prefix='Cabin')    
x_test = pd.concat([x_test, cabin_dummies_t], axis=1)

In [1070]:
# get the number of missing data points per column
missing_values_count = xy_train.isnull().sum()

# look at the # of missing points in the first ten columns
print (missing_values_count) #5 is age

# how many total missing values do we have?
total_cells = (xy_train.shape[0])
total_missing = missing_values_count[5]

# percent of data that is missing
#(total_missing/total_cells) * 100

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
Pclass_1         0
Pclass_2         0
Pclass_3         0
Title            0
Title_Master     0
Title_Miss       0
Title_Mr         0
Title_Mrs        0
Title_Officer    0
Title_Royalty    0
FamilyS          0
FamilySize       0
Singleton        0
SmallFamily      0
LargeFamily      0
Embarked_C       0
Embarked_Q       0
Embarked_S       0
Cabin_A          0
Cabin_B          0
Cabin_C          0
Cabin_D          0
Cabin_E          0
Cabin_F          0
Cabin_G          0
Cabin_T          0
Cabin_U          0
dtype: int64


In [1035]:
#play with embarked
xy_train['Embarked'].fillna('S', inplace = True)
xy_train.isnull().sum().sort_values(ascending = False)

# dummy encoding ...
Embarked_tr = pd.get_dummies(xy_train['Embarked'], prefix='Embarked')    
xy_train = pd.concat([xy_train, Embarked_tr], axis=1)

Embarked_t = pd.get_dummies(x_test['Embarked'], prefix='Embarked')    
x_test = pd.concat([x_test, Embarked_t], axis=1)

In [822]:
from sklearn.preprocessing import StandardScaler

In [828]:
#test with pclass
xy_train['Pclass_fit'] = StandardScaler().fit_transform(xy_train['Pclass'].values.reshape(-1, 1))
x_test['Pclass_fit'] = StandardScaler().fit_transform(x_test['Pclass'].values.reshape(-1, 1))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [1067]:
xy_train.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,...,Title_Officer,Title_Royalty,FamilyS,FamilySize,Singleton,SmallFamily,LargeFamily,Embarked_C,Embarked_Q,Embarked_S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,...,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.647587,0.364228,0.523008,0.381594,32.204208,0.242424,0.206510,...,0.020202,0.005612,1.904602,1.904602,0.602694,0.327722,0.069585,0.188552,0.086420,0.725028
std,257.353842,0.486592,0.836071,0.477990,0.168690,1.102743,0.806057,49.693429,0.428790,0.405028,...,0.140770,0.074743,1.613459,1.613459,0.489615,0.469646,0.254589,0.391372,0.281141,0.446751
min,1.000000,0.000000,1.000000,0.000000,0.005250,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,0.262500,0.000000,0.000000,7.910400,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,1.000000,0.325000,0.000000,0.000000,14.454200,0.000000,0.000000,...,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,668.500000,1.000000,3.000000,1.000000,0.459375,1.000000,0.000000,31.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,3.000000,1.000000,1.000000,8.000000,6.000000,512.329200,1.000000,1.000000,...,1.000000,1.000000,11.000000,11.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [1068]:
xy_train.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Title_Officer,Title_Royalty,FamilyS,FamilySize,Singleton,SmallFamily,LargeFamily,Embarked_C,Embarked_Q,Embarked_S
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",0,0.6875,0,0,248706,16.0000,...,0,0,1,1,1,0,0,0,0,1
729,730,0,3,"Ilmakangas, Miss. Pieta Sofia",0,0.3125,1,0,STON/O2. 3101271,7.9250,...,0,0,2,2,0,1,0,0,0,1
376,377,1,3,"Landergren, Miss. Aurora Adelia",0,0.2750,0,0,C 7077,7.2500,...,0,0,1,1,1,0,0,0,0,1
620,621,0,3,"Yasbeck, Mr. Antoni",1,0.3375,1,0,2659,14.4542,...,0,0,2,2,0,1,0,1,0,0
832,833,0,3,"Saad, Mr. Amin",1,0.3250,0,0,2671,7.2292,...,0,0,1,1,1,0,0,1,0,0
516,517,1,2,"Lemore, Mrs. (Amelia Milley)",0,0.4250,0,0,C.A. 34260,10.5000,...,0,0,1,1,1,0,0,0,0,1
246,247,0,3,"Lindahl, Miss. Agda Thorilda Viktoria",0,0.3125,0,0,347071,7.7750,...,0,0,1,1,1,0,0,0,0,1
416,417,1,2,"Drew, Mrs. James Vivian (Lulu Thorne Christian)",0,0.4250,1,1,28220,32.5000,...,0,0,3,3,0,1,0,0,0,1
277,278,0,2,"Parkes, Mr. Francis ""Frank""",1,0.3875,0,0,239853,0.0000,...,0,0,1,1,1,0,0,0,0,1
468,469,0,3,"Scanlan, Mr. James",1,0.3250,0,0,36209,7.7250,...,0,0,1,1,1,0,0,0,1,0


In [1071]:
class TitanicDataset(Dataset):
    """ Titanic dataset."""

    # Initialize your data, download, etc.
    def __init__(self):

        #Train data
        self.len = xy_train.shape[0]
        #self.pt_x_data = torch.from_numpy(np.array(xy_train.loc[:, ['Cabin_C','Cabin_D','Cabin_E','Cabin_F','Cabin_G','Age', 'Sex', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Parch', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs','Pclass_3', 'Title_Officer', 'Title_Royalty', 'FamilySize', 'Singleton', 'SmallFamily', 'LargeFamily', 'Fare']].values, dtype='float')).float()
        self.pt_x_data = torch.from_numpy(np.array(xy_train.loc[:, ['Pclass_1', 'Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S',  'Cabin_C','Cabin_U','Title_Mrs','Title_Miss','Title_Mr', 'Title_Master',  'Sex',   'Singleton', 'SmallFamily', 'LargeFamily']].values, dtype='float')).float()
        #self.pt_x_data = torch.from_numpy(np.array(xy_train.loc[:, ['1','2','3']].values, dtype='float')).float()
        self.y_data = np.array(xy_train.loc[:, ['Survived']].values, dtype='float')
        self.pt_y_data = torch.from_numpy(self.y_data).float()
        
        #Test data
        #self.pt_x_data_test = torch.from_numpy(np.array(x_test.loc[:, ['Age', 'Sex', 'Pclass_1', 'Pclass_2', 'Pclass_3','Parch', 'Title_Master','Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Officer', 'Title_Royalty', 'FamilySize','Pclass_3', 'Singleton', 'SmallFamily', 'LargeFamily', 'Fare']].values, dtype='float')).float()
        self.pt_x_data_test = torch.from_numpy(np.array(x_test.loc[:, [ 'Pclass_1','Pclass_3', 'Embarked_C', 'Embarked_Q', 'Embarked_S',  'Cabin_C','Cabin_U','Title_Mrs','Title_Miss', 'Title_Mr', 'Title_Master',  'Sex',  'Singleton', 'SmallFamily', 'LargeFamily']].values, dtype='float')).float()
        self.y_data_test = np.array(validation_set.loc[:, ['survived']].values, dtype='float')
        self.pt_y_data_test = torch.from_numpy(self.y_data_test).float()
        
        
    def __getitem__(self, index):
        return self.pt_x_data[index], self.pt_y_data[index]

    def __len__(self):
        return self.len


dataset = TitanicDataset()

In [1072]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=50,
                          shuffle=True,
                          num_workers=0)

In [1073]:
#type(train_loader)
print (dataset.pt_x_data.shape)
print (dataset.y_data.shape)
print (dataset.pt_y_data.shape)

#print (dataset.pt_x_data)
#print (dataset.y_data)


torch.Size([891, 15])
(891, 1)
torch.Size([891, 1])


In [1074]:
class Model(torch.nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.L1 = torch.nn.Linear(15, 270)
        self.L2 = torch.nn.Linear(270, 1)
        #self.L3 = torch.nn.Linear(50, 1)

        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        #out1 = self.sigmoid(self.l1(x))
        #out2 = self.sigmoid(self.l2(out1))
        #y_pred = self.sigmoid(self.l3(out2))
        
        #out1 = self.relu(self.L1(x))
        
        out1 = self.L1(x)
        out1 = F.dropout(out1, p=0.1)
        out1 = self.relu(out1)
        
        #out2 = self.L2(out1)
        #out2 = F.dropout(out2, p=0.1)
        #out2 = self.relu(out2)
        
        y_pred = self.sigmoid(self.L2(out1))
        
        
                
        return y_pred

# our model
model = Model()

In [1075]:
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.BCELoss(size_average=True)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='elementwise_mean' instead.
  warnings.warn(warning.format(ret))


In [1076]:
# Training loop
model.train()

for epoch in range(100):
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs)

        # Compute and print loss
        loss = criterion(y_pred, labels)
        print(epoch, i, loss.data[0])

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 0 tensor(0.6926)
0 1 tensor(0.5881)
0 2 tensor(0.4584)
0 3 tensor(0.6672)
0 4 tensor(0.5146)
0 5 tensor(0.5861)
0 6 tensor(0.4107)
0 7 tensor(0.4849)
0 8 tensor(0.4236)
0 9 tensor(0.4517)
0 10 tensor(0.4382)
0 11 tensor(0.4963)
0 12 tensor(0.3747)
0 13 tensor(0.3313)
0 14 tensor(0.3506)
0 15 tensor(0.5563)
0 16 tensor(0.5451)
0 17 tensor(0.4124)
1 0 tensor(0.3954)
1 1 tensor(0.3814)
1 2 tensor(0.3767)
1 3 tensor(0.2995)
1 4 tensor(0.4237)
1 5 tensor(0.3476)
1 6 tensor(0.6849)
1 7 tensor(0.4845)
1 8 tensor(0.4667)
1 9 tensor(0.3812)
1 10 tensor(0.3722)
1 11 tensor(0.3241)
1 12 tensor(0.3710)
1 13 tensor(0.5318)
1 14 tensor(0.3560)
1 15 tensor(0.3131)
1 16 tensor(0.5959)
1 17 tensor(0.2401)
2 0 tensor(0.3301)
2 1 tensor(0.3540)
2 2 tensor(0.4299)
2 3 tensor(0.5204)
2 4 tensor(0.5275)
2 5 tensor(0.3758)
2 6 tensor(0.3200)
2 7 tensor(0.4661)
2 8 tensor(0.3964)
2 9 tensor(0.3777)
2 10 tensor(0.4330)
2 11 tensor(0.3397)
2 12 tensor(0.3751)
2 13 tensor(0.3329)
2 14 tensor(0.3889)
2 15 tenso

24 11 tensor(0.3836)
24 12 tensor(0.3285)
24 13 tensor(0.5068)
24 14 tensor(0.3864)
24 15 tensor(0.3414)
24 16 tensor(0.3861)
24 17 tensor(0.3507)
25 0 tensor(0.4441)
25 1 tensor(0.3642)
25 2 tensor(0.3619)
25 3 tensor(0.4288)
25 4 tensor(0.3162)
25 5 tensor(0.2855)
25 6 tensor(0.3953)
25 7 tensor(0.3285)
25 8 tensor(0.3398)
25 9 tensor(0.5469)
25 10 tensor(0.2126)
25 11 tensor(0.2790)
25 12 tensor(0.3733)
25 13 tensor(0.4087)
25 14 tensor(0.3869)
25 15 tensor(0.4458)
25 16 tensor(0.3066)
25 17 tensor(0.3713)
26 0 tensor(0.3227)
26 1 tensor(0.4161)
26 2 tensor(0.3425)
26 3 tensor(0.3710)
26 4 tensor(0.3484)
26 5 tensor(0.2967)
26 6 tensor(0.2751)
26 7 tensor(0.4907)
26 8 tensor(0.2790)
26 9 tensor(0.2811)
26 10 tensor(0.4478)
26 11 tensor(0.3646)
26 12 tensor(0.4706)
26 13 tensor(0.3550)
26 14 tensor(0.2670)
26 15 tensor(0.4216)
26 16 tensor(0.3808)
26 17 tensor(0.4681)
27 0 tensor(0.3481)
27 1 tensor(0.4337)
27 2 tensor(0.5098)
27 3 tensor(0.3818)
27 4 tensor(0.2378)
27 5 tensor(0.216

48 14 tensor(0.3511)
48 15 tensor(0.2754)
48 16 tensor(0.4725)
48 17 tensor(0.5618)
49 0 tensor(0.3682)
49 1 tensor(0.2028)
49 2 tensor(0.3196)
49 3 tensor(0.3919)
49 4 tensor(0.3940)
49 5 tensor(0.3835)
49 6 tensor(0.3272)
49 7 tensor(0.2171)
49 8 tensor(0.4637)
49 9 tensor(0.3107)
49 10 tensor(0.3306)
49 11 tensor(0.3897)
49 12 tensor(0.3644)
49 13 tensor(0.4119)
49 14 tensor(0.3709)
49 15 tensor(0.4473)
49 16 tensor(0.4182)
49 17 tensor(0.3481)
50 0 tensor(0.2956)
50 1 tensor(0.3840)
50 2 tensor(0.3939)
50 3 tensor(0.3738)
50 4 tensor(0.3127)
50 5 tensor(0.3635)
50 6 tensor(0.3938)
50 7 tensor(0.2390)
50 8 tensor(0.3409)
50 9 tensor(0.3558)
50 10 tensor(0.3096)
50 11 tensor(0.3717)
50 12 tensor(0.4103)
50 13 tensor(0.5654)
50 14 tensor(0.3199)
50 15 tensor(0.4204)
50 16 tensor(0.3462)
50 17 tensor(0.3247)
51 0 tensor(0.1637)
51 1 tensor(0.2560)
51 2 tensor(0.3867)
51 3 tensor(0.3627)
51 4 tensor(0.3845)
51 5 tensor(0.3338)
51 6 tensor(0.5066)
51 7 tensor(0.3489)
51 8 tensor(0.2670)


72 17 tensor(0.3446)
73 0 tensor(0.4009)
73 1 tensor(0.2834)
73 2 tensor(0.3970)
73 3 tensor(0.3544)
73 4 tensor(0.3774)
73 5 tensor(0.3186)
73 6 tensor(0.5265)
73 7 tensor(0.3878)
73 8 tensor(0.2290)
73 9 tensor(0.4423)
73 10 tensor(0.2533)
73 11 tensor(0.2729)
73 12 tensor(0.4806)
73 13 tensor(0.4231)
73 14 tensor(0.4434)
73 15 tensor(0.2923)
73 16 tensor(0.2642)
73 17 tensor(0.2968)
74 0 tensor(0.2364)
74 1 tensor(0.3131)
74 2 tensor(0.3529)
74 3 tensor(0.3776)
74 4 tensor(0.2735)
74 5 tensor(0.3358)
74 6 tensor(0.2078)
74 7 tensor(0.4505)
74 8 tensor(0.3285)
74 9 tensor(0.3320)
74 10 tensor(0.3922)
74 11 tensor(0.4529)
74 12 tensor(0.3506)
74 13 tensor(0.5300)
74 14 tensor(0.3708)
74 15 tensor(0.3171)
74 16 tensor(0.3448)
74 17 tensor(0.5327)
75 0 tensor(0.2523)
75 1 tensor(0.4215)
75 2 tensor(0.3930)
75 3 tensor(0.2379)
75 4 tensor(0.5438)
75 5 tensor(0.3074)
75 6 tensor(0.4356)
75 7 tensor(0.3859)
75 8 tensor(0.3680)
75 9 tensor(0.3339)
75 10 tensor(0.2805)
75 11 tensor(0.2558)
7

95 15 tensor(0.4961)
95 16 tensor(0.3399)
95 17 tensor(0.3678)
96 0 tensor(0.3203)
96 1 tensor(0.3549)
96 2 tensor(0.2237)
96 3 tensor(0.3408)
96 4 tensor(0.4503)
96 5 tensor(0.4491)
96 6 tensor(0.2915)
96 7 tensor(0.2818)
96 8 tensor(0.3181)
96 9 tensor(0.3771)
96 10 tensor(0.3220)
96 11 tensor(0.4054)
96 12 tensor(0.3246)
96 13 tensor(0.5101)
96 14 tensor(0.2960)
96 15 tensor(0.2548)
96 16 tensor(0.4119)
96 17 tensor(0.5075)
97 0 tensor(0.3039)
97 1 tensor(0.4199)
97 2 tensor(0.3300)
97 3 tensor(0.2819)
97 4 tensor(0.4500)
97 5 tensor(0.4790)
97 6 tensor(0.3351)
97 7 tensor(0.2800)
97 8 tensor(0.3684)
97 9 tensor(0.3105)
97 10 tensor(0.4225)
97 11 tensor(0.4548)
97 12 tensor(0.3045)
97 13 tensor(0.2655)
97 14 tensor(0.3116)
97 15 tensor(0.4514)
97 16 tensor(0.3713)
97 17 tensor(0.2316)
98 0 tensor(0.2791)
98 1 tensor(0.2745)
98 2 tensor(0.2548)
98 3 tensor(0.5364)
98 4 tensor(0.4204)
98 5 tensor(0.2677)
98 6 tensor(0.3232)
98 7 tensor(0.3972)
98 8 tensor(0.4768)
98 9 tensor(0.4482)
9

In [ ]:
#this will be last mini batch
print (y_pred.shape)

In [1091]:
# Train and Test Set Predictions
print ("train set accuracy = " + str (accuracy(dataset.pt_x_data, dataset.y_data)))
print ("test set accuracy = " + str (accuracy(dataset.pt_x_data_test, dataset.y_data_test)))

train set accuracy = 0.8507295173961841
test set accuracy = 0.7320574162679426


In [1132]:
#show bad test rows
model.eval()


prediction = model(dataset.pt_x_data_test)

np_prediction = (prediction.detach().numpy())
np_prediction_rd = np.round(np_prediction)
print (np_prediction_rd.shape)

results = pd.DataFrame(np_prediction)
results['pred_rd'] = np_prediction_rd
results['survived'] = validation_set['survived']
results['name'] = validation_set['Name']


results

(418, 1)


,0,pred_rd,survived,name
0,0.070318,0.0,0,"Kelly, Mr. James"
1,0.514159,1.0,1,"Wilkes, Mrs. James (Ellen Needs)"
2,0.041517,0.0,0,"Myles, Mr. Thomas Francis"
3,0.124743,0.0,0,"Wirz, Mr. Albert"
4,0.514159,1.0,1,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)"
5,0.124743,0.0,1,"Svensson, Mr. Johan Cervin"
6,0.740660,1.0,0,"Connolly, Miss. Kate"
7,0.031375,0.0,1,"Caldwell, Mr. Albert Francis"
8,0.976825,1.0,1,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)"
9,0.061415,0.0,0,"Davies, Mr. John Samuel"


In [1078]:
#Run test set through
model.eval()
test_pred = model(dataset.pt_x_data_test)
print (test_pred.shape)
print (type(test_pred))

#Add prediction col to test panda
#to_np = np.round(test_pred.detach().numpy())


#to_np = (test_pred.detach().numpy())
to_np = np.where((test_pred.detach().numpy())>0.5,1.0,0)


to_np = to_np.astype(int)
x_test['Survived'] = to_np


#Export to csv for submission
x_test.to_csv('mix45a50_PClass13_parch_sib_miss_MRS_MR_MASter_sex_cabCU_port-270mod-dropout.csv', columns = ['PassengerId','Survived'], index = False)


torch.Size([418, 1])
<class 'torch.Tensor'>


In [531]:
print (test_pred[73])

tensor([0.5141], grad_fn=<SelectBackward>)


In [1090]:
def accuracy(X, Y):
    """
    This function is used to measure accuracy of the model.
    
    Arguments:
    X -- data set of examples you would like to label
    Y -- labels
    
    Returns:
    p -- predictions for the given dataset X
    """
    model.eval()
    prediction = model(X)
    #print (prediction)
    np_prediction = np.round(prediction.detach().numpy())
    predictions_correct = np.equal(np_prediction, Y)
    
    p = np.mean(predictions_correct)
    
    
    return p

In [ ]:
......................Extra Stuff...........
convertfunc = lambda x: float(x.strip("%"))/100
test = np.genfromtxt('./data/titanic/train.csv', delimiter=',', skip_header=1, usecols = (0,1,2,4))

#aai = a.iloc[:, [0,3]].values
train_x = a.loc[:, ['PassengerId', 'Pclass', 'Sex']].values
train_y = a.loc[:, ['Survived']].values

In [ ]:
print ("pt_x_data: " + str (dataset.pt_x_data.shape))
print ("pt_y_data: " + str (dataset.pt_y_data.shape))
print ("y_data: " + str (dataset.y_data.shape))

In [ ]:
print (type (train_x))
print (type (train_y))

print (train_x)
print (train_y)

In [ ]:
#Read in debug file
debug_vals = np.genfromtxt('./data/titanic/debug_justPID.csv', delimiter=',', dtype=np.float32, skip_header=1)

#convert to correct format
#debug_x_data = torch.from_numpy(np.array(xy_train.loc[:, ['PassengerId', 'Pclass', 'Sex']].values, dtype='float')).float()
#debug_y_data = np.array(xy_train.loc[:, ['Survived']].values, dtype='float')

debug_x_data = torch.from_numpy(debug_vals[:, 0:-1])
debug_y_data = np.array(debug_vals[:, [-1]])
print (debug_x_data)
print (debug_y_data)

print ("debug set accuracy = " + str (accuracy(debug_x_data,debug_y_data)))

In [ ]:
a = np.array([10,-1])
b = np.array([10,1])



In [ ]:
np.dot(a.T,b)